In [7]:
import numpy as np
import os
import torch

import sys
from pathlib import Path
parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))

from eval_scannet.openclip_encoder import OpenCLIPNetwork

import faiss

from PIL import Image
import cv2

In [5]:
index = faiss.read_index("/home/kjs/project/iclr25/lerf/index_1105_128_1024.faiss")

device = "cuda"
clip_model = OpenCLIPNetwork(device)

In [73]:
img = cv2.imread("/home/ugkim/vlm/pqgs/output/scene_dense_pth/pqqs_1_pq_openclip_topk45_weight_128/train/ours_None/gt/00011.png")
img = np.array(img) / 255
img = torch.from_numpy(img).permute((2, 0, 1)).float()
img = img.unsqueeze(0)
img = img.to(device)

In [74]:
with torch.no_grad():
    feat = clip_model.process(img).half()
    print(feat.shape)
    feat = clip_model.model.encode_image(feat)
feat = feat.cpu().numpy().astype("float32")
feat = feat / np.linalg.norm(feat, axis=1, keepdims=True)

torch.Size([1, 3, 224, 224])


In [75]:
feat[..., :10], feat.shape

(array([[ 0.07348449,  0.00628038, -0.08114807,  0.0211823 ,  0.0279327 ,
         -0.00105027, -0.10320774,  0.02596308,  0.02299076, -0.01672381]],
       dtype=float32),
 (1, 512))

In [76]:
codes = index.sa_encode(feat)

In [77]:
rec = index.sa_decode(codes)

In [78]:
rec /= np.linalg.norm(rec, axis=1, keepdims=True)

In [97]:
sim = float((feat @ rec.T).squeeze())
print(f"Cosine similarity: {sim:.4f}")

Cosine similarity: 0.8495


In [98]:
text_feat = clip_model.encode_text("sofa", device=device)
text_feat = text_feat.detach().cpu().numpy().astype("float32")

In [99]:
text_feat = text_feat / np.linalg.norm(text_feat, axis=1, keepdims=True)

In [100]:
text_codes = index.sa_encode(text_feat)
text_rec = index.sa_decode(text_codes)
text_rec /= np.linalg.norm(text_rec, axis=1, keepdims=True)

In [101]:
text_sim = float((text_feat @ text_rec.T).squeeze())
print(f"Cosine similarity: {text_sim:.4f}")

Cosine similarity: 0.6541
